In [1]:
import pandas as pd
# import tqdm to track a progress 
from tqdm import tqdm
# Ensure tqdm progress bar works with pandas
tqdm.pandas()
import itertools
import numpy as np

In [2]:
base_df = pd.read_csv('./Output/Task1-2.csv', parse_dates = ['apdedateq', 'startdate'])

In [3]:
base_df.head()

,gvkey,datadate,year,tic,conm,datacqtr,startdate,apdedateq,add1,add2,add3,add4,addzip,zip,cbsa1,cbsa2,cbsa3,cbsa4
0,1004,2015-02-28,2014,AIR,AAR CORP,2015Q1,2014-12-01,2015-02-28,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
1,1004,2015-05-31,2014,AIR,AAR CORP,2015Q2,2015-03-01,2015-05-31,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
2,1004,2015-08-31,2015,AIR,AAR CORP,2015Q3,2015-06-01,2015-08-31,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
3,1004,2015-11-30,2015,AIR,AAR CORP,2015Q4,2015-09-01,2015-11-30,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
4,1004,2016-02-29,2015,AIR,AAR CORP,2016Q1,2015-12-01,2016-02-29,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN


In [4]:
## 각 회사 (위치 : cbsa) 마다 각 row의 startdate ~ apdedateq 사이에 일어난 
# 총 protest 의 수, protestor의 수
# BigEvent 별로 aggregate (0 의 경우 don't count)
# issues 별로 aggregate

In [4]:
# The dayfirst=True argument ensures that the date parsing respects the day-first format (%d/%m/%Y) 
ccc = pd.read_csv('./Data/task3/ccc_compiled_2017-2020_bigeventscoded.csv', encoding = 'latin-1', low_memory= False, parse_dates= ['date'], dayfirst = True)

In [7]:
# Preprocessing from CCC
# - size_mean : NA -> 0 으로 변경 DONE
# - CBSA 99999 -> Drop CHECKED
# - multiple CBSA can be matched with ONE Zip code
# - if bigevent : NaN -> 0 으로 변경 DONE

In [5]:
# size mean NA 값 -> 0으로 변경
ccc['size_mean'] = ccc['size_mean'].fillna(0)

In [6]:
# bigevent na값 -> 0으로 변경
ccc['bigevent'] = ccc['bigevent'].fillna(0)

In [7]:
# Split the 'issues' column by ';' and expand into multiple columns
ccc['issues'] = ccc['issues'].str.split(';').apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)

In [8]:
ccc['issues'] = ccc['issues'].apply(lambda x: x if isinstance(x, list) else [])

In [9]:
ccc['issues']

0                                               [military]
1                                               [military]
2        [banking and finance, economy, energy, environ...
3                            [environment, guns, military]
4                                               [military]
                               ...                        
72069                              [democracy, presidency]
72070                                   [criminal justice]
72071                                   [criminal justice]
72072                          [economy, policing, racism]
72073                                   [policing, racism]
Name: issues, Length: 72074, dtype: object

In [10]:
# cbsa 99999 인 rows 없음
ccc[ccc['cbsa'] == 99999]

,date,bigevent,bydate,locality,state,location_detail,online,type,title,macroevent,...,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty,_merge1,_merge2,_merge3,_merge4,zipcode


In [11]:
ccc

,date,bigevent,bydate,locality,state,location_detail,online,type,title,macroevent,...,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty,_merge1,_merge2,_merge3,_merge4,zipcode
0,2017-01-01,0.0,1,Washington,DC,Lafayette Square Park,0.0,vigil,White House Peace Vigil,NaN,...,District of Columbia,District of Columbia,11.0,1.0,Central,Matched (3),NaN,NaN,NaN,NaN
1,2017-01-01,0.0,1,Mankato,MN,NaN,0.0,vigil,NaN,NaN,...,Blue Earth County,Minnesota,27.0,13.0,Central,Matched (3),NaN,NaN,NaN,NaN
2,2017-01-01,0.0,1,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,NaN,...,Hennepin County,Minnesota,27.0,53.0,Central,Matched (3),NaN,NaN,NaN,NaN
3,2017-01-01,0.0,1,Little Compton,RI,Town Green,0.0,vigil,NaN,NaN,...,Newport County,Rhode Island,44.0,5.0,Central,Matched (3),NaN,NaN,NaN,NaN
4,2017-01-01,0.0,1,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,NaN,...,Anderson County,Tennessee,47.0,1.0,Central,Matched (3),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72069,2020-12-31,0.0,4,San Antonio,TX,The Alamo,0.0,rally,NaN,NaN,...,Bexar County,Texas,48.0,29.0,Central,NaN,NaN,NaN,Matched (3),NaN
72070,2020-12-31,0.0,4,San Antonio,TX,Piazza Italia Park,0.0,demonstration,NaN,NaN,...,Bexar County,Texas,48.0,29.0,Central,NaN,NaN,NaN,Matched (3),NaN
72071,2020-12-31,0.0,4,Norfolk,VA,Macarthur Square,0.0,demonstration,NaN,NaN,...,Norfolk city,Virginia,51.0,710.0,Central,NaN,NaN,NaN,Matched (3),NaN
72072,2020-12-31,0.0,4,Seattle,WA,Cal Anderson Park; King County Youth Services ...,0.0,demonstration,NaN,NaN,...,King County,Washington,53.0,33.0,Central,NaN,NaN,NaN,Matched (3),NaN


In [12]:
base_df.iloc[10]

gvkey                                            1004
datadate                                   2017-08-31
year                                             2017
tic                                               AIR
conm                                         AAR CORP
datacqtr                                       2017Q3
startdate                         2017-06-01 00:00:00
apdedateq                         2017-08-31 00:00:00
add1         One AAR Place, 1100 North Wood Dale Road
add2                                              NaN
add3                                              NaN
add4                                              NaN
addzip                                          60191
zip                                           60191.0
cbsa1                                         16980.0
cbsa2                                             NaN
cbsa3                                             NaN
cbsa4                                             NaN
Name: 10, dtype: object

In [13]:
ccc.columns

Index(['date', 'bigevent', 'bydate', 'locality', 'state', 'location_detail',
       'online', 'type', 'title', 'macroevent', 'actors', 'organizations',
       'participants', 'claims', 'valence', 'issues', 'issues_major',
       'size_text', 'size_low', 'size_high', 'size_mean', 'size_cat',
       'arrests', 'arrests_any', 'injuries_crowd', 'injuries_crowd_any',
       'injuries_police', 'injuries_police_any', 'property_damage',
       'property_damage_any', 'chemical_agents', 'participant_measures',
       'police_measures', 'participant_deaths', 'police_deaths', 'source_1',
       'source_2', 'source_3', 'source_4', 'source_5', 'source_6', 'source_7',
       'source_8', 'source_9', 'source_10', 'source_11', 'source_12',
       'source_13', 'source_14', 'source_15', 'source_16', 'source_17',
       'source_18', 'source_19', 'source_20', 'source_21', 'source_22',
       'source_23', 'source_24', 'source_25', 'source_26', 'source_27',
       'source_28', 'source_29', 'source_30', 'notes

In [14]:
base_df['startdate']

0       2014-12-01
1       2015-03-01
2       2015-06-01
3       2015-09-01
4       2015-12-01
           ...    
35227   2020-10-01
35228   2021-01-01
35229   2021-04-01
35230   2021-07-01
35231   2021-10-01
Name: startdate, Length: 35232, dtype: datetime64[ns]

In [15]:
base_df['apdedateq']

0       2015-02-28
1       2015-05-31
2       2015-08-31
3       2015-11-30
4       2016-02-29
           ...    
35227   2020-12-31
35228   2021-03-31
35229   2021-06-30
35230   2021-09-30
35231   2021-12-31
Name: apdedateq, Length: 35232, dtype: datetime64[ns]

In [16]:

base_df

,gvkey,datadate,year,tic,conm,datacqtr,startdate,apdedateq,add1,add2,add3,add4,addzip,zip,cbsa1,cbsa2,cbsa3,cbsa4
0,1004,2015-02-28,2014,AIR,AAR CORP,2015Q1,2014-12-01,2015-02-28,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
1,1004,2015-05-31,2014,AIR,AAR CORP,2015Q2,2015-03-01,2015-05-31,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
2,1004,2015-08-31,2015,AIR,AAR CORP,2015Q3,2015-06-01,2015-08-31,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
3,1004,2015-11-30,2015,AIR,AAR CORP,2015Q4,2015-09-01,2015-11-30,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
4,1004,2016-02-29,2015,AIR,AAR CORP,2016Q1,2015-12-01,2016-02-29,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35227,328795,2020-12-31,2020,ACA,ARCOSA INC,2020Q4,2020-10-01,2020-12-31,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,19100.0,NaN,NaN,NaN
35228,328795,2021-03-31,2021,ACA,ARCOSA INC,2021Q1,2021-01-01,2021-03-31,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,NaN,NaN,NaN,NaN
35229,328795,2021-06-30,2021,ACA,ARCOSA INC,2021Q2,2021-04-01,2021-06-30,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,NaN,NaN,NaN,NaN
35230,328795,2021-09-30,2021,ACA,ARCOSA INC,2021Q3,2021-07-01,2021-09-30,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,NaN,NaN,NaN,NaN


In [23]:
base_df['startdate_m1'] =  base_df['startdate'] - pd.DateOffset(months=1)
base_df['apdedateq_m1'] = base_df['apdedateq'] - pd.DateOffset(months=1)

In [24]:
base_df

,gvkey,datadate,year,tic,conm,datacqtr,startdate,apdedateq,add1,add2,add3,add4,addzip,zip,cbsa1,cbsa2,cbsa3,cbsa4,startdate_m1,apdedateq_m1
0,1004,2015-02-28,2014,AIR,AAR CORP,2015Q1,2014-12-01,2015-02-28,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN,2014-11-01,2015-01-28
1,1004,2015-05-31,2014,AIR,AAR CORP,2015Q2,2015-03-01,2015-05-31,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN,2015-02-01,2015-04-30
2,1004,2015-08-31,2015,AIR,AAR CORP,2015Q3,2015-06-01,2015-08-31,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN,2015-05-01,2015-07-31
3,1004,2015-11-30,2015,AIR,AAR CORP,2015Q4,2015-09-01,2015-11-30,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN,2015-08-01,2015-10-30
4,1004,2016-02-29,2015,AIR,AAR CORP,2016Q1,2015-12-01,2016-02-29,"One AAR Place, 1100 North Wood Dale Road",NaN,NaN,NaN,60191,60191.0,NaN,NaN,NaN,NaN,2015-11-01,2016-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35227,328795,2020-12-31,2020,ACA,ARCOSA INC,2020Q4,2020-10-01,2020-12-31,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,19100.0,NaN,NaN,NaN,2020-09-01,2020-11-30
35228,328795,2021-03-31,2021,ACA,ARCOSA INC,2021Q1,2021-01-01,2021-03-31,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,NaN,NaN,NaN,NaN,2020-12-01,2021-02-28
35229,328795,2021-06-30,2021,ACA,ARCOSA INC,2021Q2,2021-04-01,2021-06-30,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,NaN,NaN,NaN,NaN,2021-03-01,2021-05-30
35230,328795,2021-09-30,2021,ACA,ARCOSA INC,2021Q3,2021-07-01,2021-09-30,"500 North Akard Street, Suite 400",NaN,NaN,NaN,75201,75201.0,NaN,NaN,NaN,NaN,2021-06-01,2021-08-30


In [25]:
# Create a function to count cases in ccc that match the conditions in base_df
def count_cases(row, ccc_df):
    
    aggregation_dict = dict()

    #1st ver
    # start_date = row['startdate']
    # end_date = row['apdedateq']

    #2nd ver
    start_date = row['startdate_m1']
    end_date = row['apdedateq_m1']
    
    # Get the list of cbsa codes from the row, filtering out None or NaN values
    cbsa_codes = [row[f'cbsa{i}'] for i in range(1, 5) if pd.notna(row[f'cbsa{i}'])]
    for index, cbsa_code in enumerate(cbsa_codes,1) :

        # Filter the ccc DataFrame for rows that match the date range and cbsa codes
        filtered_cases = ccc_df[(ccc_df['date'] >= start_date) & 
                                (ccc_df['date'] <= end_date) & 
                                (ccc_df['cbsa'] == cbsa_code)]

        aggregation_dict[f'cbsa{index}_all_prt'] = len(filtered_cases)
        aggregation_dict[f'cbsa{index}_all_par'] = filtered_cases['size_mean'].sum()
        
        # Aggregate Bigevent
        bigevents_cases = filtered_cases.groupby('bigevent').size().reset_index(name='cases')
        bigevent_ppl = filtered_cases.groupby('bigevent')['size_mean'].sum().reset_index(name = 'people')
        bigevents = bigevents_cases.merge(bigevent_ppl)[1:]
        for ntp in bigevents.itertuples():
            aggregation_dict[f'cbsa{index}_big{ntp.bigevent}_prt'] = ntp.cases
            aggregation_dict[f'cbsa{index}_big{ntp.bigevent}_par'] = ntp.people


        # Define a mapping of issue keywords to their corresponding keys in the aggregation
        issue_mapping = {
            'racism': ('racism_prt', 'racism_par'),
            'environment': ('env_prt', 'env_par'),
            'guns': ('gun_prt', 'gun_par'),
            "women's rights": ('gen_prt', 'gen_par'),
            'reproductive rights': ('gen_prt', 'gen_par'),
            'sexual violence': ('gen_prt', 'gen_par')
        }

        # Initialize a dictionary to hold the counts and sums
        small_event_aggregation = {
            'racism_prt': 0, 'racism_par': 0,
            'env_prt': 0, 'env_par': 0,
            'gun_prt': 0, 'gun_par': 0,
            'gen_prt': 0, 'gen_par': 0
        }

        # Loop through the filtered cases to aggregate the small events
        for values in filtered_cases.itertuples():
            issues = values.issues
            
            # Check each issue in the mapping
            for issue, (prt_key, par_key) in issue_mapping.items():
                if issue in issues:
                    small_event_aggregation[prt_key] += 1
                    small_event_aggregation[par_key] += values.size_mean

        # Add non-zero values to the aggregation_dict
        for key, value in small_event_aggregation.items():
            if value > 0:
                aggregation_dict[f'cbsa{index}_{key}'] = value

        # # Aggregate SmallEvents
        # racism_prt = 0
        # racism_par = 0
        # env_prt = 0
        # env_par = 0
        # gun_prt = 0
        # gun_par = 0
        # gen_prt = 0
        # gen_par = 0

        # for idx, values in filtered_cases.iterrows():
        #     issues = values['issues']
        #     print(issues)
        #     if 'racism' in issues:
        #         racism_prt += 1
        #         racism_par += values['size_mean']
        #     if 'environment' in issues:
        #         env_prt += 1
        #         env_par += values['size_mean']
        #     if 'guns' in issues:
        #         gun_prt += 1
        #         gun_par += values['size_mean']
        #     if "women's rights" in issues:
        #         gen_prt += 1
        #         gen_par += values['size_mean']
        #     if 'reproductive rights' in issues :
        #         print('yes')
        #         gen_prt += 1
        #         gen_par += values['size_mean']
        #     if 'sexual violence' in issues:
        #         print('yes')
        #         gen_prt += 1
        #         gen_par += values['size_mean']
        
        # print('gen', gen_prt, gen_par)
        
        # if racism_prt > 0 :
        #     aggregation_dict[f'cbsa{index}_racism_prt'] = racism_prt
        #     aggregation_dict[f'cbsa{index}_racism_par'] = racism_par

        # if env_prt > 0 :
        #     aggregation_dict[f'cbsa{index}_env_prt'] = env_prt
        #     aggregation_dict[f'cbsa{index}_env_par'] = env_par

        # if gun_prt > 0 :
        #     aggregation_dict[f'cbsa{index}_gun_prt'] = gun_prt
        #     aggregation_dict[f'cbsa{index}_gun_par'] = gun_par

        # if gen_prt > 0 :
        #     aggregation_dict[f'cbsa{index}_gen_prt'] = gen_prt
        #     aggregation_dict[f'cbsa{index}_gen_par'] = gen_par
            

    return aggregation_dict

In [26]:
# Apply the function to each row in base_df and create a new column for the count of cases
base_df['case_count'] = base_df.progress_apply(count_cases, axis=1, ccc_df=ccc)
# case_count = base_df[49:50].progress_apply(count_cases, axis=1, ccc_df=ccc)

100%|██████████| 35232/35232 [02:39<00:00, 221.11it/s]


In [188]:
# Apply the function to each row in base_df and create a new column for the count of cases
# base_df['case_count'] = base_df.progress_apply(count_cases, axis=1, ccc_df=ccc)
# case_count = base_df[48:49].progress_apply(count_cases, axis=1, ccc_df=ccc)

100%|██████████| 1/1 [00:00<00:00, 41.39it/s]

["women's rights"]
["women's rights"]
["women's rights"]
['reproductive rights']
yes
["women's rights"]
['healthcare']
['policing', 'racism']
['reproductive rights']
yes
['policing', 'racism']
['covid', 'economy']
['covid']
['policing', 'racism']
['policing', 'racism']
['racism', 'religion']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['policing', 'racism']
['military', 'sexual violence']
yes
['policing', 'presidency', 'racism']
['policing', 

In [27]:
# check

ccc[(ccc['cbsa'] == 16980)
    & (ccc['date'] >= (np.datetime64('2018-09-01')))
    & (ccc['date'] <= (np.datetime64('2018-11-30')))]

,date,bigevent,bydate,locality,state,location_detail,online,type,title,macroevent,...,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty,_merge1,_merge2,_merge3,_merge4,zipcode
28408,2018-09-03,83.0,3,Chicago,IL,near O'Hare Airport,0.0,protest,NaN,NaN,...,Cook County,Illinois,17.0,31.0,Central,NaN,NaN,Matched (3),NaN,NaN
28453,2018-09-05,0.0,3,Chicago,IL,outside the Leighton Criminal Court Building,0.0,rally,NaN,NaN,...,Cook County,Illinois,17.0,31.0,Central,NaN,NaN,Matched (3),NaN,NaN
28619,2018-09-08,0.0,3,Elmhurst,IL,Elmhurst Public Library,0.0,rally,NaN,NaN,...,DuPage County,Illinois,17.0,43.0,Central,NaN,NaN,Matched (3),NaN,NaN
28621,2018-09-08,0.0,3,Mundelein,IL,Fremont Township Administration Building Commu...,0.0,rally,NaN,NaN,...,Lake County,Illinois,17.0,97.0,Central,NaN,NaN,Matched (3),NaN,NaN
28624,2018-09-08,0.0,3,Elwood,IL,O'Connor Park,0.0,rally,NaN,NaN,...,Will County,Illinois,17.0,197.0,Central,NaN,NaN,Matched (3),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31701,2018-11-13,0.0,3,Midlothian,IL,outside Midlothian Police Department,0.0,protest,NaN,NaN,...,Cook County,Illinois,17.0,31.0,Central,NaN,NaN,Matched (3),NaN,NaN
31711,2018-11-14,0.0,3,Chicago,IL,the capitol building,0.0,protest,NaN,NaN,...,Cook County,Illinois,17.0,31.0,Central,NaN,NaN,Matched (3),NaN,NaN
31712,2018-11-14,0.0,3,Algonquin,IL,Outside Algonquin township,0.0,protest,NaN,NaN,...,McHenry County,Illinois,17.0,111.0,Central,NaN,NaN,Matched (3),NaN,NaN
31762,2018-11-17,10.0,3,Chicago,IL,Hyatt Regency Chicago,0.0,protest,NaN,NaN,...,Cook County,Illinois,17.0,31.0,Central,NaN,NaN,Matched (3),NaN,NaN


In [28]:
# check

import re

# Function to find the maximum 'bigN' key in the dictionary
def find_max_bigN(d):
    # Find all keys that match the pattern 'big' followed by a number (e.g., 'big1', 'big2')
    big_keys = [int(re.search(r'big(\d+)', k).group(1)) for k in d.keys() if 'big' in k]
    return max(big_keys) if big_keys else None

# Apply the function to each row and collect the results
bigN_max = base_df['case_count'].apply(find_max_bigN)

# Find the maximum bigN across all rows
overall_max_bigN = bigN_max.max()

# Display the maximum bigN found across the dataset
print(f"Maximum 'bigN' across the dataset: big{overall_max_bigN}")

Maximum 'bigN' across the dataset: big271.0


In [29]:
# CHECK
base_df[base_df['cbsa4'].notna()][['case_count']].iloc[12].values[0]

{'cbsa1_all_prt': 1,
 'cbsa1_all_par': 250.0,
 'cbsa1_racism_prt': 1,
 'cbsa1_racism_par': 250.0,
 'cbsa2_all_prt': 13,
 'cbsa2_all_par': 1700.0,
 'cbsa2_big30.0_prt': 2,
 'cbsa2_big30.0_par': 30.0,
 'cbsa2_big32.0_prt': 4,
 'cbsa2_big32.0_par': 586.0,
 'cbsa2_racism_prt': 4,
 'cbsa2_racism_par': 586.0,
 'cbsa3_all_prt': 0,
 'cbsa3_all_par': 0.0,
 'cbsa4_all_prt': 5,
 'cbsa4_all_par': 750.0,
 'cbsa4_big31.0_prt': 1,
 'cbsa4_big31.0_par': 100.0,
 'cbsa4_big32.0_prt': 1,
 'cbsa4_big32.0_par': 300.0,
 'cbsa4_racism_prt': 1,
 'cbsa4_racism_par': 300.0}

## Values in Dictionary format in case count -> flatten, stacking on the side

In [30]:
case_flattened = pd.json_normalize(base_df['case_count']).replace({}, np.nan)

In [31]:
final_df = pd.concat([base_df, case_flattened], axis=1)

In [32]:
final_df.fillna(np.nan, inplace = True)

In [33]:
final_df.drop(columns = ['case_count'], inplace=True)

In [34]:
final_df.iloc[15]['cbsa1_all_par']

12449.0

In [36]:
final_df

,gvkey,datadate,year,tic,conm,datacqtr,startdate,apdedateq,add1,add2,...,cbsa3_big29.0_prt,cbsa3_big29.0_par,cbsa3_big30.0_prt,cbsa3_big30.0_par,cbsa3_big35.0_prt,cbsa3_big35.0_par,cbsa2_big13.0_prt,cbsa2_big13.0_par,cbsa4_env_prt,cbsa4_env_par
0,1004,2015-02-28,2014,AIR,AAR CORP,2015Q1,2014-12-01,2015-02-28,"One AAR Place, 1100 North Wood Dale Road",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1004,2015-05-31,2014,AIR,AAR CORP,2015Q2,2015-03-01,2015-05-31,"One AAR Place, 1100 North Wood Dale Road",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004,2015-08-31,2015,AIR,AAR CORP,2015Q3,2015-06-01,2015-08-31,"One AAR Place, 1100 North Wood Dale Road",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,2015-11-30,2015,AIR,AAR CORP,2015Q4,2015-09-01,2015-11-30,"One AAR Place, 1100 North Wood Dale Road",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,2016-02-29,2015,AIR,AAR CORP,2016Q1,2015-12-01,2016-02-29,"One AAR Place, 1100 North Wood Dale Road",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35227,328795,2020-12-31,2020,ACA,ARCOSA INC,2020Q4,2020-10-01,2020-12-31,"500 North Akard Street, Suite 400",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35228,328795,2021-03-31,2021,ACA,ARCOSA INC,2021Q1,2021-01-01,2021-03-31,"500 North Akard Street, Suite 400",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35229,328795,2021-06-30,2021,ACA,ARCOSA INC,2021Q2,2021-04-01,2021-06-30,"500 North Akard Street, Suite 400",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35230,328795,2021-09-30,2021,ACA,ARCOSA INC,2021Q3,2021-07-01,2021-09-30,"500 North Akard Street, Suite 400",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
final_df.to_csv('./Output/Task3-2.csv', index = False)